In [ ]:
import os
import numpy as np
import pandas as pd

import tensorflow_hub as hub

In [ ]:
dir_name = '/project/data_preprocessed_csv/'

In [ ]:
df_metadata = pd.read_csv("/project/preprocessing/metadata.csv", index_col=0)

In [ ]:
cases = df_metadata.groupby('case').case.count().index.values
print(cases)

In [ ]:
case = cases[2]
print(case)

In [ ]:
filenames = df_metadata.loc[
    df_metadata.case == case, "filename"
    ].tolist()
print(filenames)

In [ ]:
# produce frame containing questions and answers from all depositions from the chosen case
df_aq = pd.DataFrame(columns=["text", "text_type", "filename"])
for filename in filenames:
    df_individual_full = pd.read_csv(dir_name + filename[:-3] + 'csv', index_col=0)
    
    indices = df_individual_full.text_type.isin(["a", "q"])
    
    df_individual_aq = pd.DataFrame(columns=["text", "text_type", "filename"])
    df_individual_aq["text"] = df_individual_full.loc[indices, "text"]
    df_individual_aq["text_type"] = df_individual_full.loc[indices, "text_type"]
    df_individual_aq["filename"] = filename[:-4]

    df_aq = pd.concat([df_aq, df_individual_aq], axis=0, ignore_index=True)

# for each question, determine if it was answered
# if yes, add that answer to new column
indices = (df_aq.text_type == 'q') & (df_aq.text_type.shift(-1) == 'a')
indices_a = indices.shift(1)
indices_a[0] = False
df_aq.loc[indices, 'answer'] = df_aq.loc[indices_a, 'text'].values

# create subframe of questions
df_q = df_aq.loc[df_aq.text_type == 'q', ['text', 'answer', 'filename']]
df_q.head(10)

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
df_q['vectors'] = embed(df_q.text).numpy().tolist()
df_q.head()

In [ ]:
question = 'what is your name'

In [ ]:
question_vector = embed([question])[0].numpy()
df_q['similarity'] = np.array(df_q.vectors.tolist()) @ question_vector.T
df_q.sort_values(by="similarity", axis=0, ascending=False, inplace=True)

In [ ]:
num = 10
for i in range(num):
    index = df_q.index[i]
    similarity = df_q.loc[index, "similarity"]
    question = df_q.loc[index, "text"]
    answer = df_q.loc[index, "answer"]
    filename = df_q.loc[index, "filename"]

    print(f"Question similarity score: {similarity}")
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print(f"Deposition: {filename}")
    print(f"Index in df_aq: {index}")
    print("=" * 10)